<a href="https://colab.research.google.com/github/bdtranter/HW-56/blob/main/Ben_Cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define hyperparameters
input_size = 32 * 32 * 3  # CIFAR-10 images are 32x32 with 3 color channels
hidden_size = 256  # Number of neurons in hidden layer
num_classes = 10  # CIFAR-10 has 10 classes
learning_rate = 0.001
num_epochs = 10
batch_size = 64

Project To-DO
-Implement Manual FCNN and Relu- Done


In [ ]:
def get_data_loaders():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [ ]:
class FullyConnectedNN:
    def __init__(self, input_size, hidden_size, num_classes):
        # Manually initializing weights and biases
        self.fc1_weight = torch.randn(hidden_size, input_size) * 0.01
        self.fc1_bias = torch.zeros(hidden_size)
        self.fc2_weight = torch.randn(num_classes, hidden_size) * 0.01
        self.fc2_bias = torch.zeros(num_classes)

    def relu(self, x):
        return (x > 0).float() * x  # Manually implemented ReLU

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = torch.matmul(x, self.fc1_weight.T) + self.fc1_bias  # First linear layer
        x = self.relu(x)  # ReLU activation
        x = torch.matmul(x, self.fc2_weight.T) + self.fc2_bias  # Second linear layer (logits)
        return x  # No softmax, raw logits are returned

In [ ]:
############################################
#         Training Function (Manual)       #
############################################
def train_model(model, train_loader, criterion, device, lr=1e-3):
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # ------------------------------------------------------
        # 1) MANUAL FORWARD PASS (instead of model.forward(images))
        # ------------------------------------------------------
        # Flatten: (N, 3, 32, 32) => (N, 3072)
        x = images.view(images.size(0), -1)

        # First linear layer: x @ W^T + b
        x = torch.matmul(x, model.fc1_weight.T) + model.fc1_bias
        # ReLU activation
        x = (x > 0).float() * x

        # Second linear layer
        outputs = torch.matmul(x, model.fc2_weight.T) + model.fc2_bias

        # ------------------------------------------------------
        # 2) Compute Loss
        # ------------------------------------------------------
        loss = criterion(outputs, labels)

        # ------------------------------------------------------
        # 3) Zero Gradients & Backprop
        # ------------------------------------------------------
        if model.fc1_weight.grad is not None:
            model.fc1_weight.grad.zero_()
            model.fc1_bias.grad.zero_()
            model.fc2_weight.grad.zero_()
            model.fc2_bias.grad.zero_()

        loss.backward()

        # ------------------------------------------------------
        # 4) Manual Gradient Descent Step
        # ------------------------------------------------------
        with torch.no_grad():
            model.fc1_weight -= lr * model.fc1_weight.grad
            model.fc1_bias   -= lr * model.fc1_bias.grad
            model.fc2_weight -= lr * model.fc2_weight.grad
            model.fc2_bias   -= lr * model.fc2_bias.grad

        # ------------------------------------------------------
        # 5) Track Statistics
        # ------------------------------------------------------
        train_loss += loss.item()
        _, predicted = outputs.max(dim=1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = train_loss / (total // batch_size)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy



In [ ]:
############################################
#        Evaluation Function (Manual)      #
############################################
def evaluate_model(model, test_loader, criterion, device):
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # -------------------------------
            # 1) MANUAL FORWARD PASS
            # -------------------------------
            x = images.view(images.size(0), -1)
            x = torch.matmul(x, model.fc1_weight.T) + model.fc1_bias
            x = (x > 0).float() * x  # ReLU
            outputs = torch.matmul(x, model.fc2_weight.T) + model.fc2_bias

            # -------------------------------
            # 2) Compute Loss
            # -------------------------------
            loss = criterion(outputs, labels)

            # -------------------------------
            # 3) Track Statistics
            # -------------------------------
            test_loss += loss.item()
            _, predicted = outputs.max(dim=1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = test_loss / (total // batch_size)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy


In [ ]:
# Main execution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader = get_data_loaders()
model = FullyConnectedNN(input_size, hidden_size, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam([model.fc1_weight, model.fc1_bias, model.fc2_weight, model.fc2_bias], lr=learning_rate)

train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')


In [ ]:
# Plot results
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Test Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, num_epochs+1), test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Training and Test Accuracy')

plt.show()
